In [ ]:
import json
from pyspark.sql.functions import explode, split
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, TimestampType, MapType, ArrayType, BooleanType
#Convert data Unix timestamp to datetime
from pyspark.sql.functions import from_unixtime
from pyspark.sql.functions import countDistinct
from pyspark.sql.functions import count
from pyspark.sql.functions import desc
import pyspark.sql.functions as f


game_api_raw = spark \
  .read \
  .format("kafka") \
  .option("kafka.bootstrap.servers", "kafka:29092") \
  .option("subscribe","events") \
  .option("startingOffsets", "earliest") \
  .option("endingOffsets", "latest") \
  .load() 
game_api_raw.cache()
game_api_raw.show()

In [ ]:
game_api_raw.printSchema()
game_api_raw.count()

In [ ]:
#Get the 'value' field and cast it as a string 
game_api_string = game_api_raw.select(game_api_raw.value.cast('string'))
game_api_string.show()

In [ ]:
game_api_df = game_api_string.rdd.map(lambda x: json.loads(x.value)).toDF()

game_api_df.printSchema()
game_api_df.show(100)

In [ ]:
#Save all of the responses with all collumns to a parque file
game_api_df.write.mode("overwrite").parquet("/tmp/game/all_api_requests")

#Drop the collum with the request headers as they are not need for the downstream analysis 
game_api_df_light = game_api_df.drop('request_headers')

#Save the purchases and sales to seperate files for analysis 
game_api_df_light.filter(game_api_df_light.event_type=="sell_item").write.mode("overwrite").parquet("/tmp/game/sell_api")
game_api_df_light.filter(game_api_df_light.event_type=="purchase_item").write.mode("overwrite").parquet("/tmp/game/purchase_api")

In [2]:
#Create a function that can be used to list the contents of a directory in Hadoop
import sys
def list_hadoop_files(path):
    hadoop = sc._jvm.org.apache.hadoop
    fs = hadoop.fs.FileSystem
    conf = hadoop.conf.Configuration() 
    fs_path = hadoop.fs.Path(path)

    try:
        for f in fs.get(conf).listStatus(fs_path):
            print(f.getPath(), f.getLen())
    except:
        print("Path '{}' does not exist.".format(path))

list_hadoop_files('/tmp/game/')

file:/tmp/game/purchase_api 4096
file:/tmp/game/all_api_requests 4096
file:/tmp/game/sell_api 4096


In [ ]:
import json

#Read events from kafka
game_api_raw = spark \
  .read \
  .format("kafka") \
  .option("kafka.bootstrap.servers", "kafka:29092") \
  .option("subscribe","events") \
  .option("startingOffsets", "earliest") \
  .option("endingOffsets", "latest") \
  .load() 
game_api_raw.cache()

#Select the 'value' collumn as a string so it can be converted to a json object 
game_api_string = game_api_raw.select(game_api_raw.value.cast('string'))
game_api_df = game_api_string.rdd.map(lambda x: json.loads(x.value)).toDF()

#Save all of the responses with all collumns to a parque file
game_api_df.write.mode("overwrite").parquet("/tmp/game/all_api_requests")

#Drop the collum with the request headers as they are not need for the downstream analysis 
game_api_df_light = game_api_df.drop('request_headers')

#Save the purchases and sales to seperate files for analysis 
game_api_df_light.filter(game_api_df_light.event_type=="sell_item").write.mode("overwrite").parquet("/tmp/game/sell_api")
game_api_df_light.filter(game_api_df_light.event_type=="purchase_item").write.mode("overwrite").parquet("/tmp/game/purchase_api")

In [ ]:
import sc._jvm.org.apache.hadoop
